# Multi-Agent Collaboration 
---

### What is Multi-Agent Collaboration?
Multi-Agent Collaboration refers to the process where multiple autonomous agents—each capable of independent decision-making—work together to achieve common or complementary objectives. This concept is widely used in fields like artificial intelligence, robotics, distributed computing, and simulation, and it involves several key aspects:

- **Effective Communication and Coordination**:
Agents exchange information and align their actions to collectively achieve a goal, ensuring that tasks are organized and synchronized.

- **Autonomous, Distributed Decision-Making**:
Each agent operates independently, making local decisions while contributing to a broader strategy, which enhances flexibility and fault tolerance.

- **Adaptive Task Specialization**:
Agents focus on specific roles or subtasks based on their capabilities, and they adjust their strategies through iterative feedback, leading to improved overall performance.


### Key Advantages
- **Efficiency Through Task Specialization**:
By assigning specific roles to different agents, the system can handle complex tasks in parallel. This specialization allows each agent to focus on its area of expertise, resulting in faster and more effective problem-solving.

- **Scalability and Flexibility**:
AutoGen's structured communication and dynamic task allocation enable the system to scale easily. It can adapt to varying project complexities by adding or reassigning agents as needed, ensuring that the collaboration remains robust even as demands change.

- **Enhanced Iterative Refinement**:
The framework’s built-in feedback loops and iterative dialogue facilitate continuous improvement. Agents can refine their outputs based on real-time feedback, leading to more accurate and cohesive final results.

**Reference**
- [AutoGen paper: Enabling Next-Gen LLM Applications via Multi-Agent Conversation](https://arxiv.org/abs/2308.08155)
- [Multi-Agent Collabration Concept](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#network) 

In [1]:
import asyncio
from typing import Annotated
from dotenv import load_dotenv
import json
import os
from enum import Enum
from semantic_kernel import Kernel
from semantic_kernel.kernel_pydantic import KernelBaseSettings
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents import AuthorRole
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread



kernel = Kernel()



class Service(Enum):
    """Attributes:
    OpenAI (str): Represents the OpenAI service.
    AzureOpenAI (str): Represents the Azure OpenAI service.
    HuggingFace (str): Represents the HuggingFace service.
    """

    OpenAI = "openai"
    AzureOpenAI = "azureopenai"
    HuggingFace = "huggingface"

class ServiceSettings(KernelBaseSettings):
    """The Learn Resources Service Settings.

    The settings are first loaded from environment variables. If the
    environment variables are not found, the settings can be loaded from a .env file with the
    encoding 'utf-8' as default or the specific encoding. If the settings are not found in the
    .env file, the settings are ignored; however, validation will fail alerting that the settings
    are missing.

    Args:
        global_llm_service (str | None): The LLM service to use for the samples, either "OpenAI" or "AzureOpenAI"
            If not provided, defaults to "AzureOpenAI".
    """

    global_llm_service: str | None = None
    
load_dotenv(override=True)

True

We will load our settings and get the LLM service to use for the notebook.

In [2]:
service_settings = ServiceSettings()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


We now configure our Chat Completion service on the kernel.

In [3]:
# Remove all services so that this cell can be re-run without restarting the kernel
kernel.remove_all_services()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

<br>


## 🧪 Step 1. Define the Agentic Architecture
- Before building the agentic pipeline, we need to design the message, topic, agent and message routing logic. 
- You should define the terminate condition for the pipeline.

### Define Agents

In the next section we will define the agents that will be used in the travel planning team.

<br>

## 🧪 Step 2. Execute the Agents TeamChat

### Execute the group chat with the termination condition

In [4]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "approved" in history[-1].content.lower()


REVIEWER_NAME = "ArtDirector"
REVIEWER_INSTRUCTIONS = """
You are an art director who has opinions about copywriting born of a love for David Ogilvy.
The goal is to determine if the given copy is acceptable to print.
If so, state that it is approved.  Do not use the word "approve" unless you are giving approval.
If not, provide insight on how to refine suggested copy without example.
"""

COPYWRITER_NAME = "CopyWriter"
COPYWRITER_INSTRUCTIONS = """
You are a copywriter with ten years of experience and are known for brevity and a dry humor.
The goal is to refine and decide on the single best copy as an expert in the field.
Only provide a single proposal per response.
You're laser focused on the goal at hand.
Don't waste time with chit chat.
Consider suggestions when refining an idea.
"""

TASK = "a slogan for a new line of electric cars."

ai_agent_settings = AzureAIAgentSettings()

async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(credential=creds) as client,
):
    # 1. Create the reviewer agent on the Azure AI agent service
    reviewer_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=REVIEWER_NAME,
        instructions=REVIEWER_INSTRUCTIONS,
    )

    # 2. Create a Semantic Kernel agent for the reviewer Azure AI agent
    agent_reviewer = AzureAIAgent(
        client=client,
        definition=reviewer_agent_definition,
    )

    # 3. Create the copy writer agent on the Azure AI agent service
    copy_writer_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=COPYWRITER_NAME,
        instructions=COPYWRITER_INSTRUCTIONS,
    )

    # 4. Create a Semantic Kernel agent for the copy writer Azure AI agent
    agent_writer = AzureAIAgent(
        client=client,
        definition=copy_writer_agent_definition,
    )

    # 5. Place the agents in a group chat with a custom termination strategy
    chat = AgentGroupChat(
        agents=[agent_writer, agent_reviewer],
        termination_strategy=ApprovalTerminationStrategy(agents=[agent_reviewer], maximum_iterations=10),
    )
    
    try:
        # 6. Add the task as a message to the group chat
        await chat.add_chat_message(message=TASK)
        print(f"# {AuthorRole.USER}: '{TASK}'")
        # 7. Invoke the chat
        async for content in chat.invoke():
            print(f"# {content.role} - {content.name or '*'}: '{content.content}'")
    finally:
        # 8. Cleanup: Delete the agents
        await chat.reset()
        await client.agents.delete_agent(agent_reviewer.id)
        await client.agents.delete_agent(agent_writer.id)

    

# AuthorRole.USER: 'a slogan for a new line of electric cars.'
# AuthorRole.ASSISTANT - CopyWriter: '"Charge Forward."'
# AuthorRole.ASSISTANT - ArtDirector: 'This is succinct and hints at both progress and the electric nature of the product. However, it could be more evocative or emotional to connect deeper with the audience. Consider an angle that emphasizes innovation, sustainability, or the joy of driving. Refine for a stronger emotional punch.'
# AuthorRole.ASSISTANT - CopyWriter: '"Charge Forward."'
# AuthorRole.ASSISTANT - ArtDirector: 'This is succinct and hints at both progress and the electric nature of the product. However, it could be more evocative or emotional to connect deeper with the audience. Consider an angle that emphasizes innovation, sustainability, or the joy of driving. Refine for a stronger emotional punch.'
# AuthorRole.ASSISTANT - CopyWriter: '"Charge Forward."'
# AuthorRole.ASSISTANT - ArtDirector: 'This is succinct and hints at both progress and the electri

## 🧪 Case 3 group chat with ChatCompletionAgent
---

In [5]:
TASK = "a slogan for a new line of electric cars."

def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id))
    return kernel

# 1. Create the reviewer agent based on the chat completion service
agent_reviewer = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion("artdirector"),
    name=REVIEWER_NAME,
    instructions=REVIEWER_INSTRUCTIONS,
)

# 2. Create the copywriter agent based on the chat completion service
agent_writer = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion("copywriter"),
    name=COPYWRITER_NAME,
    instructions=COPYWRITER_INSTRUCTIONS,
)

# 3. Place the agents in a group chat with a custom termination strategy
group_chat = AgentGroupChat(
    agents=[
        agent_writer,
        agent_reviewer,
    ],
    termination_strategy=ApprovalTerminationStrategy(
        agents=[agent_reviewer],
        maximum_iterations=10,
    ),
)

# 4. Add the task as a message to the group chat
await group_chat.add_chat_message(message=TASK)
print(f"# User: {TASK}")

# 5. Invoke the chat
async for content in group_chat.invoke():
    print(f"# {content.name}: {content.content}")

# User: a slogan for a new line of electric cars.


# CopyWriter: "Drive the Future. Leave Emissions Behind."
# ArtDirector: The slogan effectively communicates the essence of the product. However, it could be refined to make it more memorable and impactful. Consider focusing on a unique selling proposition or the emotional benefits of driving electric. Aim for a shorter, punchier phrase that sticks in the mind, perhaps by incorporating a call to action or a sense of excitement about the environmental benefits. Think about how to evoke a feeling of transformation or empowerment associated with making the switch to electric.
# CopyWriter: "Plug In. Drive Clean."
# ArtDirector: The slogan is concise and delivers a clear message about the product's eco-friendly benefits. However, to enhance its appeal, you might consider adding a touch of emotional resonance or visionary language that captures the forward-thinking nature of electric driving. Aim for something that not only informs but also inspires the audience to make a change.
# CopyWrit